In [1]:
import pprint
from bscscan import BscScan
import asyncio
import nest_asyncio
import pandas as pd
import datetime
import numpy as np
from datetime import timezone

In [2]:
nest_asyncio.apply()

In [3]:
start_timestamp = int(datetime.datetime(2021,7,5,4,0,0).replace(tzinfo=timezone.utc).timestamp())
end_timestamp = int(datetime.datetime(2021,7,5,13,30,0).replace(tzinfo=timezone.utc).timestamp())

In [4]:
async with BscScan('SCSKHAVMN19QAFCNSII2N6UADH89M8TAKV') as client:
        start_block_number=int(await client.get_block_number_by_timestamp(
            timestamp=start_timestamp,
            closest="before"
        ))

In [1]:
async with BscScan('SCSKHAVMN19QAFCNSII2N6UADH89M8TAKV') as client:
        end_block_number=int(await client.get_block_number_by_timestamp(
            timestamp=end_timestamp,
            closest="before"
        ))

NameError: name 'BscScan' is not defined

In [6]:
# async with BscScan('SCSKHAVMN19QAFCNSII2N6UADH89M8TAKV') as client:
#     end_block_number=int(await client.get_proxy_block_number(),16)


In [7]:
start_block_number,end_block_number

(8851778, 8863091)

In [8]:
async def get_internal_txs(address,start_block_number,end_block_number,offset):
    async with BscScan('SCSKHAVMN19QAFCNSII2N6UADH89M8TAKV') as client:
        return await client.get_internal_txs_by_address_paginated(\
         address=address,page=1,offset=offset,\
         startblock=start_block_number,endblock=end_block_number,sort="asc")

In [9]:
Data = []
block_number = start_block_number
address = "0x0000000000000000000000000000000000001004"
offset = 10000
while block_number < end_block_number:
    try:
        data=asyncio.run(get_internal_txs(address,block_number,end_block_number,offset))
        Data.append(data)
        block_number = int(data[-1]['blockNumber']) +1
    except: 
        break
    

In [10]:
len(Data)

4

In [11]:
from itertools import chain 

In [12]:
# df = pd.DataFrame(list(chain.from_iterable(Data)),columns=["timeStamp","blockNumber","to","value"])
df = pd.DataFrame(list(chain.from_iterable(Data)),columns=["timeStamp","to","value"])
df['value'] =  df['value'].astype(float);
df['value'] = df['value']/pow(10,18)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36543 entries, 0 to 36542
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timeStamp  36543 non-null  object 
 1   to         36543 non-null  object 
 2   value      36543 non-null  float64
dtypes: float64(1), object(2)
memory usage: 856.6+ KB


In [14]:
wallet_array = df.loc[(df['value']< 0.64)&(df['value']>0.38)]['to'].unique()

In [15]:
len(wallet_array)

1591

In [16]:
new_wallet = []
for wallet in wallet_array:
    data = asyncio.run(get_internal_txs(wallet,0,end_block_number,1))
    if int(data[0]['timeStamp']) >= start_timestamp: 
        new_wallet.append(wallet)
    

In [17]:
len(new_wallet)

346

In [18]:
pumper = []
pump_wallet =[]
for wallet in new_wallet:
    try:
        async with BscScan('SCSKHAVMN19QAFCNSII2N6UADH89M8TAKV') as client:
            data = await client.get_bep20_token_transfer_events_by_address(
            address=wallet,
            startblock=0,
            endblock = end_block_number,
            sort="asc")
        if int(data[0]['timeStamp']) >= start_timestamp and data[0]['tokenName'] == data[1]['tokenName'] and data[0]['tokenName']!= 'SafeMoon' and data[0]['tokenName']!= 'Baby Doge Coin' and data[0]['tokenName']!= 'PancakeSwap Token'  :
            pumper.append([wallet,data[0]['timeStamp'],data[0]['tokenName']])
            pump_wallet.apend(wallet)
    except:
        continue
      

In [19]:
len(pumper)

20

In [20]:
pumper

[['0xc1cdbccf127c9ef72ea32a18038a7fb6ceeb37be',
  '1625384089',
  'Binance-Peg BUSD Token'],
 ['0x8dce958e629a0bafb1281e238af4c37dd67ddf3e',
  '1625376869',
  'Binance-Peg BSC-USD'],
 ['0x90bc51ca5759a53438587b6e2aade6e055a5d591', '1625374232', 'ZOLUCOIN'],
 ['0x23747b7724ad2e17c76152ab64f1f66f26279f52', '1625375276', 'LITTLE RABBIT'],
 ['0x7120e73100c8055870698f8f3787bc39a55e8594', '1625396740', 'SkullDex'],
 ['0x7886456040b6bb5114de5528ca4a52f5e708f8b4',
  '1625381506',
  'Token War Game'],
 ['0x12e0694966e1bfa1a78431c37dfda68953aa7a57', '1625384629', 'Lunar'],
 ['0x7cc6e903ba3467decabbeeaa7484c6e1cb76cff9',
  '1625382403',
  'PhoenixProtocol'],
 ['0xd69df5695fcdc05a25b0332f73224a70b464272f', '1625384992', 'BULL COIN'],
 ['0x1358c7621b5fb58f24214ec9f357c93204857faa', '1625386315', 'DeFi Coin'],
 ['0x5bd605315046081ccb37f6611d1a3f38539e5ec9',
  '1625387164',
  'Mommy Doge Coin'],
 ['0xc8157d4fc15e75c4daf8efd0065d6d86f0c3a426', '1625384767', 'xxxNifty'],
 ['0x9906d1f43ef5556c9a5a1fdf56

In [21]:
new_wallet_bep20_transfer = []
suspect = list(set(new_wallet) - set(pump_wallet))
for wallet in suspect:
    try:
        async with BscScan('SCSKHAVMN19QAFCNSII2N6UADH89M8TAKV') as client:
            data = await client.get_bep20_token_transfer_events_by_address(
            address=wallet,
            startblock=0,
            endblock = end_block_number,
            sort="asc")
            if int(data[0]['timeStamp']) >= start_timestamp and data[0]['tokenName']!= 'SafeMoon' and data[0]['tokenName']!= 'Baby Doge Coin' and data[0]['tokenName']!= 'PancakeSwap Token'  :
                new_wallet_bep20_transfer.append([wallet,data[0]['timeStamp'],data[0]['tokenName']])

    except:
        continue

In [22]:
len(new_wallet_bep20_transfer)

54

In [23]:
new_wallet_bep20_transfer

[['0xc8157d4fc15e75c4daf8efd0065d6d86f0c3a426', '1625384767', 'xxxNifty'],
 ['0x243045839daad4be9357f9ccf01b4726587fbabe', '1625402457', 'PEAKDEFI'],
 ['0xb9f8db6e80f6b42a9bbd518122c9e42ab563ee90', '1625391148', 'Diamond Hands'],
 ['0xb3741858936674d10fd11bbe37d39db8f4bebf71', '1625391103', 'Pornstar'],
 ['0xb361c0c42b1011aba56c3aad1608a5e6b7342283', '1625404673', 'Babybnb'],
 ['0x54c6b0c025b11be166443dde07f8f01c3e1740fe', '1625393211', 'Wrapped BNB'],
 ['0x28d1e012b108411e5bccf1aec8d1f0fc0b7f7506', '1625388025', 'LayBaby'],
 ['0x39d1c3804138308641295ca198281f9898595eec', '1625378417', 'Tixl Token'],
 ['0x8b86cb08b5dc04dc18cf5ae6c3f7f6c688fd72d7', '1625388835', 'MiniDOGE'],
 ['0x6441b1514a34e530d9501900e6eed249d436d781', '1625390944', 'CheeseSwap'],
 ['0x9906d1f43ef5556c9a5a1fdf568f2a044eaa50c6', '1625386639', 'Biswap'],
 ['0xd7067da8d751354c9b8b3106603f67fe3bcbbc5a', '1625382139', 'SAFEMOONmax'],
 ['0x164012319c90c1bb7503b2d84edec2d0a9cc645e', '1625381116', 'Porn Kub'],
 ['0x47bfcce62

In [24]:
async with BscScan('SCSKHAVMN19QAFCNSII2N6UADH89M8TAKV') as client:
    data=await client.get_internal_txs_by_address_paginated(
            address="0x0000000000000000000000000000000000001004",
            startblock=8737719,
            endblock=8738919,
            page = 0,
            offset = 10000,
            sort="asc"
        )

In [25]:
ts=1625026802
datetime.datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')

'2021-06-30 04:20:02'

In [26]:
async with BscScan('SCSKHAVMN19QAFCNSII2N6UADH89M8TAKV') as client:
    print(
        await client.get_proxy_transaction_count(
            address="0x90af9eae299ace375ba97b89691c828e856efad6"
        )
    )

0x1


In [27]:
new_wallet

['0x020f5b6c7fa7479e00b78fbcb36dd0b15c2e9a97',
 '0x53294ef1c08ccc425ed94c64f67966d9e26e4332',
 '0x9d7e76ec149938a9b29371946e8876a6f5760624',
 '0x3ff5acde0342d2a1f97ee99184971f3e406b97c8',
 '0xbcb73b5986d68ccfc418828868283032860cf49d',
 '0x5c5e6dbe54bd4084f0d0af871c9bbcdc0a002ebc',
 '0xc92224711a5225c24f13ce367862af61147a7dc3',
 '0x9359507ad5a0d309a5cb3ef758ecda644bae938e',
 '0x60e205a33ddea6ce398c9b1a6614acf8b8380660',
 '0xc1cdbccf127c9ef72ea32a18038a7fb6ceeb37be',
 '0xd1d860334e9b8bd86cc2d0a61f35bc67c76480c1',
 '0x0bfd19a501478e46de0340f72dd4b99230f4c170',
 '0x55f416dc95056822e3fbf354fff4671efe1bce08',
 '0xa7c1d2194d2ca82b9624bfe50a50600f9265db36',
 '0x6811cdd49cf79b2497b7d557fb59376645efe0b2',
 '0x0b75afcd90280187efc9626bd0080f078de97798',
 '0x6be29086c4971864c2e35ff0507b710ec06d5109',
 '0xc993c3d7216e0a196e3346728bb968e3a9a7fc5a',
 '0x2993e88a447db1f151d6c60a2851a1d277e78222',
 '0x823638b9d0d8f5877a6971330300eae180966841',
 '0x8325600a89159c61ec49e4768d0957b4cfc4a318',
 '0x6368c71ce

In [28]:
sus = []
pump_wallet =[]
for wallet in new_wallet:
    try:
        async with BscScan('SCSKHAVMN19QAFCNSII2N6UADH89M8TAKV') as client:
            data = await client.get_bep20_token_transfer_events_by_address(
            address=wallet,
            startblock=0,
            endblock = end_block_number,
            sort="asc")
    except:
        sus.append(wallet)


In [29]:
len(sus)

58

In [30]:
sus

['0xc92224711a5225c24f13ce367862af61147a7dc3',
 '0xa7c1d2194d2ca82b9624bfe50a50600f9265db36',
 '0x6368c71ce468086354eb91ccc3f05c13c40cf2ac',
 '0x211d6866a15de6618d99af3b917c85aafa8ebb89',
 '0x656df516518c5a2d2b8152561982d822b9e6792c',
 '0x90f0cc00418d02fbdfbfef48f6d8e045ed391d9c',
 '0x80042240718c3e1675022d4bd9071a44dc91a58a',
 '0xac2881a4d2a7d9b6bdb4cb1bce44577291acf085',
 '0xd871b15da36caf3ffe7531ac737541f87244ec38',
 '0x3cbafd9ca28f1642d964dda55cbdb6179b1466fc',
 '0x1258530f18ae058ad835743e42d16c470b3edce8',
 '0xb7b20612f76fd988bd2242f6f432d9ea03a209a9',
 '0x3622229540e6720423ecc31a139130e8502d79e2',
 '0x0e3916df0d8dd7ad926813fb4286e415e66212e7',
 '0x01c65b901f239ea7a4c3df5da64fba0015b683cd',
 '0x1520fe65948d76da2c43ac7468ac439b006fe877',
 '0xa9658d3d5628a25c62adecc17c71351fd64127aa',
 '0xfb74db1176e5b2bfc2540f9e1adc0ccf86b13b36',
 '0xee2d04a859422bc59e1f5b458045acd2f7b952b9',
 '0xd94386c3d5d3dc6e37fdc87d4e24c76d7d2aef0d',
 '0x0b3410ef3c3a20ef6586548b3a248211f15888bc',
 '0x1d7dd08ec

In [31]:
new_wallet

['0x020f5b6c7fa7479e00b78fbcb36dd0b15c2e9a97',
 '0x53294ef1c08ccc425ed94c64f67966d9e26e4332',
 '0x9d7e76ec149938a9b29371946e8876a6f5760624',
 '0x3ff5acde0342d2a1f97ee99184971f3e406b97c8',
 '0xbcb73b5986d68ccfc418828868283032860cf49d',
 '0x5c5e6dbe54bd4084f0d0af871c9bbcdc0a002ebc',
 '0xc92224711a5225c24f13ce367862af61147a7dc3',
 '0x9359507ad5a0d309a5cb3ef758ecda644bae938e',
 '0x60e205a33ddea6ce398c9b1a6614acf8b8380660',
 '0xc1cdbccf127c9ef72ea32a18038a7fb6ceeb37be',
 '0xd1d860334e9b8bd86cc2d0a61f35bc67c76480c1',
 '0x0bfd19a501478e46de0340f72dd4b99230f4c170',
 '0x55f416dc95056822e3fbf354fff4671efe1bce08',
 '0xa7c1d2194d2ca82b9624bfe50a50600f9265db36',
 '0x6811cdd49cf79b2497b7d557fb59376645efe0b2',
 '0x0b75afcd90280187efc9626bd0080f078de97798',
 '0x6be29086c4971864c2e35ff0507b710ec06d5109',
 '0xc993c3d7216e0a196e3346728bb968e3a9a7fc5a',
 '0x2993e88a447db1f151d6c60a2851a1d277e78222',
 '0x823638b9d0d8f5877a6971330300eae180966841',
 '0x8325600a89159c61ec49e4768d0957b4cfc4a318',
 '0x6368c71ce